<a href="https://colab.research.google.com/github/NAshwinKumar/credit_risk_assessment/blob/master/HGANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder 
from collections import defaultdict

In [0]:
data = pd.read_excel('https://github.com/NAshwinKumar/credit_risk_assessment/raw/master/GermanCredit.xls', sheet_name='Data')

# Exploratory Data Analysis an dPre-processing

In [3]:
print ('\033[1m' + "Number of Rows     : " +'\033[0m', data.shape[0])
print ('\033[1m' + "\nNumber of Columns  : " +'\033[0m', data.shape[1])
print ('\033[1m' + "\nMissing values     :  "+'\033[0m', data.isnull().values.any())
print('\033[1m' + "\nColumn Names\t\t\tUnique Values  \n"+'\033[0m', data.nunique(),sep="")
print('\033[1m' + "\nColumn Names\t\t\tData Types  \n"+'\033[0m', data.dtypes,sep="")

Number of Rows     :  1000

Number of Columns  :  32

Missing values     :   False

Column Names			Unique Values  
OBS#                1000
CHK_ACCT               4
DURATION              33
HISTORY                5
NEW_CAR                2
USED_CAR               2
FURNITURE              2
RADIO/TV               2
EDUCATION              2
RETRAINING             2
AMOUNT               921
SAV_ACCT               5
EMPLOYMENT             5
INSTALL_RATE           4
MALE_DIV               2
MALE_SINGLE            2
MALE_MAR_or_WID        2
CO-APPLICANT           2
GUARANTOR              2
PRESENT_RESIDENT       4
REAL_ESTATE            2
PROP_UNKN_NONE         2
AGE                   53
OTHER_INSTALL          2
RENT                   2
OWN_RES                2
NUM_CREDITS            4
JOB                    4
NUM_DEPENDENTS         2
TELEPHONE              2
FOREIGN                2
RESPONSE               2
dtype: int64

Column Names			Data Types  
OBS#                int64
CHK_ACCT         

In [0]:
data = data.drop(['OBS#'], axis=1)

## Min-max normalisation

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(data) 
data.loc[:] = scaled_values

In [6]:
data

,CHK_ACCT,DURATION,HISTORY,NEW_CAR,USED_CAR,FURNITURE,RADIO/TV,EDUCATION,RETRAINING,AMOUNT,SAV_ACCT,EMPLOYMENT,INSTALL_RATE,MALE_DIV,MALE_SINGLE,MALE_MAR_or_WID,CO-APPLICANT,GUARANTOR,PRESENT_RESIDENT,REAL_ESTATE,PROP_UNKN_NONE,AGE,OTHER_INSTALL,RENT,OWN_RES,NUM_CREDITS,JOB,NUM_DEPENDENTS,TELEPHONE,FOREIGN,RESPONSE
0,0.000000,0.029412,1.00,0.0,0.0,0.0,1.0,0.0,0.0,0.050567,1.00,1.00,1.000000,0.0,1.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.857143,0.0,0.0,1.0,0.333333,0.666667,0.0,1.0,0.0,1.0
1,0.333333,0.647059,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.313690,0.00,0.50,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,0.0,0.053571,0.0,0.0,1.0,0.000000,0.666667,0.0,0.0,0.0,0.0
2,1.000000,0.117647,1.00,0.0,0.0,0.0,0.0,1.0,0.0,0.101574,0.00,0.75,0.333333,0.0,1.0,0.0,0.0,0.0,0.666667,1.0,0.0,0.535714,0.0,0.0,1.0,0.000000,0.333333,1.0,0.0,0.0,1.0
3,0.000000,0.558824,0.50,0.0,0.0,1.0,0.0,0.0,0.0,0.419941,0.00,0.75,0.333333,0.0,1.0,0.0,0.0,1.0,1.000000,0.0,0.0,0.464286,0.0,0.0,0.0,0.000000,0.666667,1.0,0.0,0.0,1.0
4,0.000000,0.294118,0.75,1.0,0.0,0.0,0.0,0.0,0.0,0.254209,0.00,0.50,0.666667,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,1.0,0.607143,0.0,0.0,0.0,0.333333,0.666667,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.000000,0.117647,0.50,0.0,0.0,1.0,0.0,0.0,0.0,0.081765,0.00,0.75,0.666667,0.0,0.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.214286,0.0,0.0,1.0,0.000000,0.333333,0.0,0.0,0.0,1.0
996,0.000000,0.382353,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.198470,0.00,0.50,1.000000,1.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.375000,0.0,0.0,1.0,0.000000,1.000000,0.0,1.0,0.0,1.0
997,1.000000,0.117647,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.030483,0.00,1.00,1.000000,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.339286,0.0,0.0,1.0,0.000000,0.666667,0.0,0.0,0.0,1.0
998,0.000000,0.602941,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.087763,0.00,0.50,1.000000,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,1.0,0.071429,0.0,0.0,0.0,0.000000,0.666667,0.0,1.0,0.0,0.0
